In [12]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import re
from scipy.stats import spearmanr
import statsmodels.api as sm
import seaborn as sb
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [13]:
player_df = pd.read_csv("player_data.csv")
stats_df = pd.read_csv("Seasons_Stats.csv", index_col = 0)

In [14]:
stats_df.drop(stats_df[stats_df['Year'] < 2000.0].index, inplace = True) # looking at only 2000 season to present
stats_df = stats_df.dropna(how = 'all') # drop all rows with all NaN values
stats_df['Player'] = stats_df['Player'].str.rstrip('*')  # get rid of stars next to players' names
stats_df = stats_df[['Player', 'Year', "Age", "Pos", "Tm", "MP", "FG", "FGA", "FT", "FTA", "3P", "3PA", "TRB", "AST", "PF", "TOV"]]
stats_df

,Player,Year,Age,Pos,Tm,MP,FG,FGA,FT,FTA,3P,3PA,TRB,AST,PF,TOV
14470,Tariq Abdul-Wahad,2000.0,25.0,SG,TOT,1578.0,274.0,646.0,146.0,193.0,3.0,23.0,291.0,98.0,147.0,106.0
14471,Tariq Abdul-Wahad,2000.0,25.0,SG,ORL,1205.0,223.0,515.0,115.0,151.0,2.0,21.0,239.0,72.0,116.0,87.0
14472,Tariq Abdul-Wahad,2000.0,25.0,SG,DEN,373.0,51.0,131.0,31.0,42.0,1.0,2.0,52.0,26.0,31.0,19.0
14473,Shareef Abdur-Rahim,2000.0,23.0,SF,VAN,3223.0,594.0,1277.0,446.0,551.0,29.0,96.0,825.0,271.0,244.0,249.0
14474,Cory Alexander,2000.0,26.0,PG,DEN,329.0,28.0,98.0,17.0,22.0,9.0,35.0,42.0,58.0,39.0,28.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24686,Cody Zeller,2017.0,24.0,PF,CHO,1725.0,253.0,443.0,133.0,196.0,0.0,1.0,405.0,99.0,189.0,65.0
24687,Tyler Zeller,2017.0,27.0,C,BOS,525.0,78.0,158.0,22.0,39.0,0.0,1.0,124.0,42.0,61.0,20.0
24688,Stephen Zimmerman,2017.0,20.0,C,ORL,108.0,10.0,31.0,3.0,5.0,0.0,0.0,35.0,4.0,17.0,3.0
24689,Paul Zipser,2017.0,22.0,SF,CHI,843.0,88.0,221.0,31.0,40.0,33.0,99.0,125.0,36.0,78.0,40.0


In [15]:
player_df = player_df.rename(columns={"name": "Player"})
player_df.drop(columns = 'birth_date', inplace = True) # get rid of columns we don't find important
player_df.drop(columns = 'position', inplace = True)
player_df.drop(columns = 'college', inplace = True)

In [16]:
player_df.dropna(how = 'any', inplace = True)           # get rid of any row with at least one NaN value
print(player_df['height'].isnull().value_counts())      # ensure that all null values have been eliminated
print(player_df['Player'].isnull().value_counts())
print(player_df['weight'].isnull().value_counts())
print(player_df['year_start'].isnull().value_counts())
print(player_df['year_end'].isnull().value_counts())
player_df.set_index("Player", inplace = True)

player_df

False    4544
Name: height, dtype: int64
False    4544
Name: Player, dtype: int64
False    4544
Name: weight, dtype: int64
False    4544
Name: year_start, dtype: int64
False    4544
Name: year_end, dtype: int64


,year_start,year_end,height,weight
Player,,,,
Alaa Abdelnaby,1991,1995,6-10,240.0
Zaid Abdul-Aziz,1969,1978,6-9,235.0
Kareem Abdul-Jabbar,1970,1989,7-2,225.0
Mahmoud Abdul-Rauf,1991,2001,6-1,162.0
Tariq Abdul-Wahad,1998,2003,6-6,223.0
...,...,...,...,...
Ante Zizic,2018,2018,6-11,250.0
Jim Zoet,1983,1983,7-1,240.0
Bill Zopf,1971,1971,6-1,170.0


In [17]:
def height(x):         # function to turn height column into it's value in inches
    array = []  
    for i in range(len(x)):
        string = str(x[i])
        feet = float(string.split("-")[0])
        inches = float(string.split("-")[1])
        array.append(feet*12 + inches)
    return array

new_height = height(player_df["height"])  # using function on height column
player_df['height'] = new_height
player_df = player_df.rename(columns={"year_start": "Year Started", "year_end": "Year Ended", "height": "Height", "weight": "Weight"})
player_df

,Year Started,Year Ended,Height,Weight
Player,,,,
Alaa Abdelnaby,1991,1995,82.0,240.0
Zaid Abdul-Aziz,1969,1978,81.0,235.0
Kareem Abdul-Jabbar,1970,1989,86.0,225.0
Mahmoud Abdul-Rauf,1991,2001,73.0,162.0
Tariq Abdul-Wahad,1998,2003,78.0,223.0
...,...,...,...,...
Ante Zizic,2018,2018,83.0,250.0
Jim Zoet,1983,1983,85.0,240.0
Bill Zopf,1971,1971,73.0,170.0


In [18]:
final_df = pd.merge(stats_df, player_df, on = 'Player')  # merge two datasets to create final dataset

In [19]:
final_df.head()

,Player,Year,Age,Pos,Tm,MP,FG,FGA,FT,FTA,3P,3PA,TRB,AST,PF,TOV,Year Started,Year Ended,Height,Weight
0,Tariq Abdul-Wahad,2000.0,25.0,SG,TOT,1578.0,274.0,646.0,146.0,193.0,3.0,23.0,291.0,98.0,147.0,106.0,1998,2003,78.0,223.0
1,Tariq Abdul-Wahad,2000.0,25.0,SG,ORL,1205.0,223.0,515.0,115.0,151.0,2.0,21.0,239.0,72.0,116.0,87.0,1998,2003,78.0,223.0
2,Tariq Abdul-Wahad,2000.0,25.0,SG,DEN,373.0,51.0,131.0,31.0,42.0,1.0,2.0,52.0,26.0,31.0,19.0,1998,2003,78.0,223.0
3,Tariq Abdul-Wahad,2001.0,26.0,SG,DEN,420.0,43.0,111.0,21.0,36.0,4.0,10.0,59.0,22.0,54.0,34.0,1998,2003,78.0,223.0
4,Tariq Abdul-Wahad,2002.0,27.0,SG,TOT,441.0,55.0,147.0,24.0,33.0,1.0,2.0,84.0,24.0,56.0,27.0,1998,2003,78.0,223.0


In [20]:
# column created for counting years a player had been in league
final_df.insert(4, "Year in League", (final_df["Year"] + 1) - final_df["Year Started"] , True) 

In [21]:
final_df.head()

,Player,Year,Age,Pos,Year in League,Tm,MP,FG,FGA,FT,...,3P,3PA,TRB,AST,PF,TOV,Year Started,Year Ended,Height,Weight
0,Tariq Abdul-Wahad,2000.0,25.0,SG,3.0,TOT,1578.0,274.0,646.0,146.0,...,3.0,23.0,291.0,98.0,147.0,106.0,1998,2003,78.0,223.0
1,Tariq Abdul-Wahad,2000.0,25.0,SG,3.0,ORL,1205.0,223.0,515.0,115.0,...,2.0,21.0,239.0,72.0,116.0,87.0,1998,2003,78.0,223.0
2,Tariq Abdul-Wahad,2000.0,25.0,SG,3.0,DEN,373.0,51.0,131.0,31.0,...,1.0,2.0,52.0,26.0,31.0,19.0,1998,2003,78.0,223.0
3,Tariq Abdul-Wahad,2001.0,26.0,SG,4.0,DEN,420.0,43.0,111.0,21.0,...,4.0,10.0,59.0,22.0,54.0,34.0,1998,2003,78.0,223.0
4,Tariq Abdul-Wahad,2002.0,27.0,SG,5.0,TOT,441.0,55.0,147.0,24.0,...,1.0,2.0,84.0,24.0,56.0,27.0,1998,2003,78.0,223.0


In [27]:
# condition for duplicate rows in each player's seasons 
duplicate_condition = final_df.duplicated(subset=['Player', 'Year'], keep = False) 
all_dupes = final_df[duplicate_condition]   # dataset for players with multiple entries per season
no_dupes = final_df[~duplicate_condition]   # dataset for players without multiple entries per season
dropped_dupes = all_dupes[all_dupes["Tm"] == "TOT"] # drop all but TOT rows in all_dupes
nba = dropped_dupes.append(no_dupes)                # create new dataset for TOT rows and no_dupes
nba.sort_values(by=['Year', 'Player'], inplace = True)  
nba = nba[nba["Year in League"] <= 23.0] # drop all bad data in Year In League
nba = nba[nba["MP"] > 100.0] #keep only players who played more than 100 mins during any season
nba.reset_index(drop=True, inplace = True)
nba.to_csv('/Users/elena.pertsalis/Desktop/nba.csv', index=False) 
nba

,Player,Year,Age,Pos,Year in League,Tm,MP,FG,FGA,FT,...,3P,3PA,TRB,AST,PF,TOV,Year Started,Year Ended,Height,Weight
0,A.C. Green,2000.0,36.0,PF,15.0,LAL,1929.0,173.0,387.0,66.0,...,1.0,4.0,486.0,80.0,127.0,53.0,1986,2001,81.0,220.0
1,Aaron McKie,2000.0,27.0,SG,6.0,PHI,1952.0,244.0,593.0,121.0,...,44.0,121.0,246.0,240.0,194.0,113.0,1995,2007,77.0,209.0
2,Aaron Williams,2000.0,28.0,PF,7.0,WAS,1545.0,235.0,450.0,146.0,...,0.0,3.0,409.0,58.0,234.0,80.0,1994,2008,81.0,220.0
3,Adam Keefe,2000.0,29.0,C,8.0,UTA,604.0,53.0,130.0,29.0,...,0.0,1.0,136.0,34.0,90.0,46.0,1993,2001,81.0,230.0
4,Adonal Foyle,2000.0,24.0,C,3.0,GSW,1654.0,193.0,380.0,34.0,...,0.0,0.0,424.0,42.0,218.0,71.0,1998,2009,82.0,250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7307,Wilson Chandler,2017.0,29.0,SF,10.0,DEN,2197.0,433.0,939.0,141.0,...,110.0,326.0,459.0,141.0,172.0,114.0,2008,2018,80.0,225.0
7308,Yogi Ferrell,2017.0,23.0,PG,1.0,TOT,1197.0,160.0,394.0,74.0,...,68.0,176.0,111.0,172.0,90.0,70.0,2017,2018,72.0,180.0
7309,Zach LaVine,2017.0,21.0,SG,3.0,MIN,1749.0,326.0,710.0,117.0,...,120.0,310.0,159.0,139.0,104.0,85.0,2015,2017,77.0,189.0
7310,Zach Randolph,2017.0,35.0,PF,16.0,MEM,1786.0,433.0,964.0,141.0,...,21.0,94.0,598.0,122.0,136.0,99.0,2002,2018,81.0,260.0
